# Base Line - GRU classifier

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True,)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import classification_report


In [ ]:
# Funcction which takes sentence as input and gives word wise BERT embedding by summing up the last 4 hidden states

def bert_embedded(text, model):

    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    model.eval()
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    token_vecs_sum = []

    for token in token_embeddings:
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec)

    return token_vecs_sum

In [ ]:
# This path contains the CSV file

PATH = '/content/drive/MyDrive/Tweets/sent_final_preds.csv'

df = pd.read_csv(PATH, sep = ",")

df = df.sample(frac=1).reset_index(drop=True) 
df.head(2)

In [ ]:
labels = []
sentim = []

for i in range(len(df)):
    labels.append(df.Labels[i])
    if df.Sentiment_Vader[i] == 'Negative':
        sentim.append(0)
    if df.Sentiment_Vader[i] == 'Neutral':
        sentim.append(1)
    if df.Sentiment_Vader[i] == 'Positive':
        sentim.append(2)

In [ ]:
tweets = []
i = 0

for tweet in df['Tweets']:
  tweets.append(bert_embedded(tweet, model))
  i += 1

  if i % 100 == 0:
      print(i, 'done')

In [ ]:
avg_len = 0
max_len = 0

for tweet in tweets:
    avg_len += len(tweet)
    
    if len(tweet) > max_len:
        max_len = len(tweet)

avg_len = int(avg_len / len(tweets))

print(max_len)
print(avg_len)

In [ ]:
print(np.stack(tweet).shape)

In [ ]:
# Taking the maximum vector length to be 40
# So if < 40, padded with zeros
# Else if > 40, trimmed down to 40

final_vect = []

for tweet in tweets: 

    vect = np.stack(tweet)
    
    if len(vect) < 40:     
        new_vect = np.concatenate((vect, np.zeros((40 - len(vect), vect.shape[1]))))
    
    else:
        new_vect = vect[:40]

    final_vect.append(new_vect)

In [ ]:
final_stack = np.stack(final_vect)

print(final_stack.shape)

In [ ]:
label = np.array(df.Labels)

print(label.shape)

In [ ]:
batch_size = 128

trainx, testx, trainy, testy = train_test_split(final_stack, label, test_size = 0.2, random_state = 42)

train_data = TensorDataset(torch.from_numpy(trainx), torch.from_numpy(trainy))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [ ]:
# 2 layered GRU with hidden size = 256
# and dropout of 0.2

class GRUNet(nn.Module):

    def __init__(self, input_dim):

        super(GRUNet, self).__init__()

        self.gru = nn.GRU(input_dim, 128, 2, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(128, 2)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):

        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, b_size = 128):

        weight = next(self.parameters()).data
        hidden = weight.new(2, b_size, 128).zero_()

        return hidden


In [ ]:
def train(train_loader):
    
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].shape[2]


    # Instantiating the models
    model = GRUNet(input_dim)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=10e-4)
    
    model.train()

    epoch_times = []
    # Start training loop
    for epoch in range(150):
        
        t_loss = 0

        h = model.init_hidden()

        for x, label in train_loader:

            h = h.data
            model.zero_grad()
            
            out, h = model(x.float(), h)

            model.zero_grad()
            
            loss = torch.nn.functional.cross_entropy(out, label)

            loss.backward()
            optimizer.step()
            
            t_loss += loss.detach().numpy() 

        print('Epoch: ', epoch, ';  loss: ', np.round(t_loss, len(train_loader)))

    return model

In [ ]:
gru_model = train(train_loader)

In [ ]:
def evaluate(model, testx, testy):

    model.eval()
    preds = []

    for i in range(len(testx)):

        inp = torch.from_numpy(np.array(testx[i]))

        h = model.init_hidden(b_size = 1)

        inp = inp.reshape((1, inp.shape[0], inp.shape[1]))

        out, h = model(inp.float(), h)

        preds.append(torch.argmax(out))

    report = classification_report(testy, torch.tensor(preds), labels=[0, 1])
    print(report)

In [ ]:
evaluate(gru_model, testx, testy)